# Pre-procesamiento Michoacán

Importamos las librerías necesarias.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle
from itertools import combinations
from scipy import stats
from statsmodels import graphics,tsa
from statsmodels.tsa import stattools
import statsmodels.graphics.tsaplots as tsp
import statsmodels.api as sm

Leemos nuestros datos.

In [2]:
df = pd.read_csv("/home/jupyter-achacon/SS/SS-Incidencia-Delictiva/Datos/IDM_NM_sep21.csv", encoding='latin-1')

In [3]:
df.head()

,Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2,...,1,1,0,1,1,0,2,1.0,0.0,1.0
1,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,...,0,0,0,1,0,1,0,0.0,0.0,0.0
2,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,...,1,1,3,2,0,1,2,0.0,0.0,0.0
3,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,1,...,0,1,0,0,0,0,0,0.0,0.0,0.0
4,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0,...,0,0,1,0,0,0,0,0.0,0.0,0.0


Cuando hacemos el filtrado nos genera nuevos índices los cuales provoca un error en nuestra función, por lo cual hacemos uso del **reset_index** la cual nos genera una nueva columna de índices la cual también borramos.

In [4]:
df['Entidad'].unique()

array(['Aguascalientes', 'Baja California', 'Baja California Sur',
       'Campeche', 'Coahuila de Zaragoza', 'Colima', 'Chiapas',
       'Chihuahua', 'Ciudad de México', 'Durango', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán de Ocampo',
       'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla',
       'Querétaro', 'Quintana Roo', 'San Luis Potosí', 'Sinaloa',
       'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala',
       'Veracruz de Ignacio de la Llave', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [5]:
df_micho = df['Entidad'] == 'Michoacán de Ocampo'
df_micho = df[df_micho]
df_micho.reset_index(inplace = True)
df_micho = df_micho.drop(['index'], axis=1)

In [6]:
def arreglar_datos(dataset):
    df_chido_corto = dataset.iloc[:,[1,2,3,4,5,6,7,8,0]]
    
    meses = cycle(["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio",
        "Agosto","Septiembre","Octubre","Noviembre","Diciembre"])

    df_chido_chido = df_chido_corto.loc[np.repeat(df_chido_corto.index, 12)]
    df_chido_chido['Mes'] = [next(meses) for j in range(len(df_chido_chido))]
    df_chido_chido.reset_index(inplace = True)
    
    df_transpuesto = dataset.iloc[:,[9,10,11,12,13,14,15,16,17,18,19,20]].transpose()
    
    lista = []

    for i in range(len(df_chido_corto)):
        lista.append(df_transpuesto[i])

    chido = pd.concat(lista).to_list()
    df_chido_chido['Cantidad'] = chido
    
    return df_chido_chido

In [7]:
df_normalizado = arreglar_datos(df_micho)

In [8]:
df_normalizado

,index,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Año,Mes,Cantidad
0,0,16,Michoacán de Ocampo,16001,Acuitzio,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2015,Enero,0.0
1,0,16,Michoacán de Ocampo,16001,Acuitzio,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2015,Febrero,0.0
2,0,16,Michoacán de Ocampo,16001,Acuitzio,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2015,Marzo,0.0
3,0,16,Michoacán de Ocampo,16001,Acuitzio,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2015,Abril,0.0
4,0,16,Michoacán de Ocampo,16001,Acuitzio,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2015,Mayo,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
930211,77517,16,Michoacán de Ocampo,16113,José Sixto Verduzco,Otros bienes jurídicos afectados (del fuero co...,Otros delitos del Fuero Común,Otros delitos del Fuero Común,Otros delitos del Fuero Común,2021,Agosto,0.0
930212,77517,16,Michoacán de Ocampo,16113,José Sixto Verduzco,Otros bienes jurídicos afectados (del fuero co...,Otros delitos del Fuero Común,Otros delitos del Fuero Común,Otros delitos del Fuero Común,2021,Septiembre,0.0
930213,77517,16,Michoacán de Ocampo,16113,José Sixto Verduzco,Otros bienes jurídicos afectados (del fuero co...,Otros delitos del Fuero Común,Otros delitos del Fuero Común,Otros delitos del Fuero Común,2021,Octubre,NaN
930214,77517,16,Michoacán de Ocampo,16113,José Sixto Verduzco,Otros bienes jurídicos afectados (del fuero co...,Otros delitos del Fuero Común,Otros delitos del Fuero Común,Otros delitos del Fuero Común,2021,Noviembre,NaN


Obtener tuplas unicas

In [9]:
df_tuplas = pd.DataFrame(df_micho.iloc[:,[6,7,8]])
df_tuplas_unicas = df_tuplas.drop_duplicates()
df_tuplas_unicas

,Tipo de delito,Subtipo de delito,Modalidad
0,Homicidio,Homicidio doloso,Con arma de fuego
1,Homicidio,Homicidio doloso,Con arma blanca
2,Homicidio,Homicidio doloso,Con otro elemento
3,Homicidio,Homicidio doloso,No especificado
4,Homicidio,Homicidio culposo,Con arma de fuego
...,...,...,...
93,Falsificación,Falsificación,Falsificación
94,Contra el medio ambiente,Contra el medio ambiente,Contra el medio ambiente
95,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos
96,Electorales,Electorales,Electorales


In [10]:
tuplas = df_tuplas_unicas.to_numpy().tolist()
tuplas

[['Homicidio', 'Homicidio doloso', 'Con arma de fuego'],
 ['Homicidio', 'Homicidio doloso', 'Con arma blanca'],
 ['Homicidio', 'Homicidio doloso', 'Con otro elemento'],
 ['Homicidio', 'Homicidio doloso', 'No especificado'],
 ['Homicidio', 'Homicidio culposo', 'Con arma de fuego'],
 ['Homicidio', 'Homicidio culposo', 'Con arma blanca'],
 ['Homicidio', 'Homicidio culposo', 'En accidente de tránsito'],
 ['Homicidio', 'Homicidio culposo', 'Con otro elemento'],
 ['Homicidio', 'Homicidio culposo', 'No especificado'],
 ['Lesiones', 'Lesiones dolosas', 'Con arma de fuego'],
 ['Lesiones', 'Lesiones dolosas', 'Con arma blanca'],
 ['Lesiones', 'Lesiones dolosas', 'Con otro elemento'],
 ['Lesiones', 'Lesiones dolosas', 'No especificado'],
 ['Lesiones', 'Lesiones culposas', 'Con arma de fuego'],
 ['Lesiones', 'Lesiones culposas', 'Con arma blanca'],
 ['Lesiones', 'Lesiones culposas', 'En accidente de tránsito'],
 ['Lesiones', 'Lesiones culposas', 'Con otro elemento'],
 ['Lesiones', 'Lesiones culpos

Juntar en una nueva columna con mes-año.

Crear una serie de tiempo de algun tipo de delito ['Homicidio', 'Homicidio doloso', 'Con arma de fuego']

In [11]:
def armar_etiquetas_tuplas(municipio, tupla):
    return (municipio + '_' + tupla[0] + '_' + tupla[1] + '_' + tupla[2])

In [12]:
armar_etiquetas_tuplas("Morelia", tuplas[0])

'Morelia_Homicidio_Homicidio doloso_Con arma de fuego'

In [13]:
def series_de_tiempo(municipio):
    series_todas = pd.DataFrame()
    s = list([])
    for i in range(len(tuplas)):
        df_series = df_normalizado[(df_normalizado['Municipio'] == municipio) & (df_normalizado['Tipo de delito'] == tuplas[i][0])
            & (df_normalizado['Subtipo de delito'] == tuplas[i][1]) & (df_normalizado['Modalidad'] == tuplas[i][2])]
    
        series_todas[i] = df_series['Cantidad'].to_list()
        nombre_columnas = armar_etiquetas_tuplas(municipio, tuplas[i])
        s.append([nombre_columnas])
        
    meses = cycle(["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio",
        "Agosto","Septiembre","Octubre","Noviembre","Diciembre"])
    
    #series_todas.index = [next(meses) for j in range(len(series_todas))] 
    series_todas['Fecha'] = pd.date_range("2015-01-01", periods=len(series_todas), freq="MS")
    series_todas = series_todas.set_index('Fecha')
    series_todas.columns = np.array(s)
        
    return series_todas

In [14]:
df_normalizado['Municipio'].unique()

array(['Acuitzio', 'Aguililla', 'Álvaro Obregón', 'Angamacutiro',
       'Angangueo', 'Apatzingán', 'Aporo', 'Aquila', 'Ario', 'Arteaga',
       'Briseñas', 'Buenavista', 'Carácuaro', 'Coahuayana',
       'Coalcomán de Vázquez Pallares', 'Coeneo', 'Contepec', 'Copándaro',
       'Cotija', 'Cuitzeo', 'Charapan', 'Charo', 'Chavinda', 'Cherán',
       'Chilchota', 'Chinicuila', 'Chucándiro', 'Churintzio', 'Churumuco',
       'Ecuandureo', 'Epitacio Huerta', 'Erongarícuaro', 'Gabriel Zamora',
       'Hidalgo', 'La Huacana', 'Huandacareo', 'Huaniqueo', 'Huetamo',
       'Huiramba', 'Indaparapeo', 'Irimbo', 'Ixtlán', 'Jacona', 'Jiménez',
       'Jiquilpan', 'Juárez', 'Jungapeo', 'Lagunillas', 'Madero',
       'Maravatío', 'Marcos Castellanos', 'Lázaro Cárdenas', 'Morelia',
       'Morelos', 'Múgica', 'Nahuatzen', 'Nocupétaro',
       'Nuevo Parangaricutiro', 'Nuevo Urecho', 'Numarán', 'Ocampo',
       'Pajacuarán', 'Panindícuaro', 'Parácuaro', 'Paracho', 'Pátzcuaro',
       'Penjamillo', 'Pe

In [15]:
series_de_tiempo("Morelia")

,"(Morelia_Homicidio_Homicidio doloso_Con arma de fuego,)","(Morelia_Homicidio_Homicidio doloso_Con arma blanca,)","(Morelia_Homicidio_Homicidio doloso_Con otro elemento,)","(Morelia_Homicidio_Homicidio doloso_No especificado,)","(Morelia_Homicidio_Homicidio culposo_Con arma de fuego,)","(Morelia_Homicidio_Homicidio culposo_Con arma blanca,)","(Morelia_Homicidio_Homicidio culposo_En accidente de tránsito,)","(Morelia_Homicidio_Homicidio culposo_Con otro elemento,)","(Morelia_Homicidio_Homicidio culposo_No especificado,)","(Morelia_Lesiones_Lesiones dolosas_Con arma de fuego,)",...,"(Morelia_Narcomenudeo_Narcomenudeo_Narcomenudeo,)","(Morelia_Amenazas_Amenazas_Amenazas,)","(Morelia_Allanamiento de morada_Allanamiento de morada_Allanamiento de morada,)","(Morelia_Evasión de presos_Evasión de presos_Evasión de presos,)","(Morelia_Falsedad_Falsedad_Falsedad,)","(Morelia_Falsificación_Falsificación_Falsificación,)","(Morelia_Contra el medio ambiente_Contra el medio ambiente_Contra el medio ambiente,)","(Morelia_Delitos cometidos por servidores públicos_Delitos cometidos por servidores públicos_Delitos cometidos por servidores públicos,)","(Morelia_Electorales_Electorales_Electorales,)","(Morelia_Otros delitos del Fuero Común_Otros delitos del Fuero Común_Otros delitos del Fuero Común,)"
Fecha,,,,,,,,,,,,,,,,,,,,,
2015-01-01,8.0,3.0,0.0,4.0,0.0,0.0,9.0,0.0,5.0,2.0,...,30.0,14.0,2.0,0.0,2.0,5.0,1.0,9.0,0.0,65.0
2015-02-01,6.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,13.0,3.0,...,41.0,13.0,1.0,0.0,1.0,10.0,0.0,14.0,0.0,39.0
2015-03-01,10.0,5.0,1.0,1.0,0.0,0.0,7.0,6.0,9.0,6.0,...,42.0,7.0,0.0,0.0,0.0,4.0,2.0,10.0,1.0,38.0
2015-04-01,12.0,3.0,1.0,0.0,1.0,0.0,0.0,6.0,11.0,17.0,...,10.0,8.0,1.0,0.0,0.0,6.0,2.0,9.0,5.0,50.0
2015-05-01,13.0,1.0,0.0,5.0,2.0,0.0,0.0,3.0,14.0,3.0,...,6.0,9.0,0.0,1.0,0.0,0.0,1.0,8.0,5.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,32.0,0.0,1.0,4.0,0.0,0.0,10.0,0.0,0.0,10.0,...,61.0,123.0,3.0,0.0,1.0,40.0,0.0,13.0,1.0,89.0
2021-09-01,26.0,3.0,1.0,1.0,0.0,0.0,6.0,1.0,1.0,15.0,...,92.0,99.0,7.0,0.0,2.0,27.0,1.0,13.0,0.0,66.0
2021-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_normalizado['Municipio'].unique()[0]

'Acuitzio'

In [17]:
series_michoacan = series_de_tiempo(df_normalizado['Municipio'].unique()[0])

for i in df_normalizado['Municipio'].unique()[1:]:
    series_michoacan = pd.concat([series_michoacan, series_de_tiempo(i)], axis=1)

In [18]:
series_michoacan

,"(Acuitzio_Homicidio_Homicidio doloso_Con arma de fuego,)","(Acuitzio_Homicidio_Homicidio doloso_Con arma blanca,)","(Acuitzio_Homicidio_Homicidio doloso_Con otro elemento,)","(Acuitzio_Homicidio_Homicidio doloso_No especificado,)","(Acuitzio_Homicidio_Homicidio culposo_Con arma de fuego,)","(Acuitzio_Homicidio_Homicidio culposo_Con arma blanca,)","(Acuitzio_Homicidio_Homicidio culposo_En accidente de tránsito,)","(Acuitzio_Homicidio_Homicidio culposo_Con otro elemento,)","(Acuitzio_Homicidio_Homicidio culposo_No especificado,)","(Acuitzio_Lesiones_Lesiones dolosas_Con arma de fuego,)",...,"(José Sixto Verduzco_Narcomenudeo_Narcomenudeo_Narcomenudeo,)","(José Sixto Verduzco_Amenazas_Amenazas_Amenazas,)","(José Sixto Verduzco_Allanamiento de morada_Allanamiento de morada_Allanamiento de morada,)","(José Sixto Verduzco_Evasión de presos_Evasión de presos_Evasión de presos,)","(José Sixto Verduzco_Falsedad_Falsedad_Falsedad,)","(José Sixto Verduzco_Falsificación_Falsificación_Falsificación,)","(José Sixto Verduzco_Contra el medio ambiente_Contra el medio ambiente_Contra el medio ambiente,)","(José Sixto Verduzco_Delitos cometidos por servidores públicos_Delitos cometidos por servidores públicos_Delitos cometidos por servidores públicos,)","(José Sixto Verduzco_Electorales_Electorales_Electorales,)","(José Sixto Verduzco_Otros delitos del Fuero Común_Otros delitos del Fuero Común_Otros delitos del Fuero Común,)"
Fecha,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-02-01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2015-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2015-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2015-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
type(series_michoacan)

pandas.core.frame.DataFrame

Agregar una columna nueva con tipo de datos de pandas https://pandas.pydata.org/docs/user_guide/timeseries.html.

SQL-Pandas: https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [20]:
series_michoacan.to_csv('tabla_completa_michoacan.csv', index=True)

In [ ]:
titu = delitos.columns[1764]
delitos.iloc[:,1764][:-3].plot(rot=90, title=titu)

## Análisis de las series de tiempo

In [ ]:
delitos = pd.read_csv("tabla_completa.csv", index_col=0) 
delitos

In [ ]:
c = 1765
s = delitos.iloc[:,c][:-3]
plt.plot(s)
plt.title(delitos.columns[c])
plt.show()

In [ ]:
todos = delitos.iloc[:,1][:-3]
todos

ACF: la función de autocorrelación

La función de autocorrelación es una herramienta que ayuda a identificar si existe autocorrelación en su serie temporal.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
plot_acf(d_series)

PACF: la función de autocorrelación

El PACF es una alternativa al ACF. En lugar de dar las autocorrelaciones, te da la autocorrelación parcial. Esta autocorrelación se llama parcial, porque con cada paso hacia atrás en el pasado, solo se enumera la autocorrelación adicional. Esto es diferente del ACF, ya que el ACF contiene correlaciones duplicadas cuando la variabilidad puede explicarse por múltiples puntos en el tiempo.

Por ejemplo, si el valor de hoy es el mismo que el valor de ayer, pero también el mismo que el de anteayer, el ACF mostraría dos pasos altamente correlacionados. El PACF solo mostraría ayer y eliminaría anteayer.

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(d_series)